# Overview:
This jupyter notebook was used for exploring the datasets and cleaning them up so that they will be easier to work with when they are in the database. In this jupyter notebook we open `disappearance.csv` and `domestic-consumption.csv` as pandas dataframes, clean them and merge them together. We then also load `gdp_per_capita.csv` into a pandas dataframe and then clean up the data in the dataframe so that it only contains entries for countries found in the merged coffee consumption dataframe (`concat_consumption`). The cleaned dataframes are then saved as .csv files in the `cleaned-datasets` directory, which are then loaded into the `coffeegdp_db` database using sqlalchemy.

In [ ]:
# import dependencies
import requests
import pandas as pd
from sqlalchemy import create_engine, inspect

# Important:
### Make sure your **username**, **password,** and **port** are all correct below before running the code in this notebook !!

In [ ]:
# Create database connection
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'coffeegdp_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

# Extract:
Load our datasets into dataframes to be cleaned and then loaded.

In [ ]:
# read disappearance.csv as dataframe
import_consumption_path = "datasets/disappearance.csv"

import_consumption_data = pd.read_csv(import_consumption_path)
import_consumption_data

In [ ]:
# set path for exporting country consumption and reading it in dataframe
export_consumption_path = "datasets/domestic-consumption.csv"

export_consumption_data = pd.read_csv(export_consumption_path)
export_consumption_data

In [ ]:
# set path for gdp_per_capita.csv and open it in a dataframe
gdp_path = "datasets/gdp_per_capita.csv"
gdp_data = pd.read_csv(gdp_path)

gdp_data

# Transform:
Take newly created dataframes and clean them so they are easier to work with for any would be analysts using these datasets in the future.

In [ ]:
# rename disappearance to 'import' and 'disappearance' column to 'country'
import_renamed = import_consumption_data.rename(columns ={"disappearance" : "Country"})
import_renamed.head()

In [ ]:
# i originally was using this to change null values to 0, but i figured that for the intergrity of the data and for future analysis it would be better to keep the null entries as null instead of changing them to 0
cleaned_import = import_renamed
cleaned_import

In [ ]:
# rename 'domestic_consumption' to 'country'
export_renamed = export_consumption_data.rename(columns = {'domestic_consumption' : 'Country'})
export_renamed

In [ ]:
# merge import and export consumption dataframes on 'country' in 'concat_consumption'
frames = [cleaned_import, export_renamed]

concat_consumption = pd.concat(frames)
concat_consumption = concat_consumption.set_index('Country')
concat_consumption = concat_consumption.reset_index()

concat_consumption

In [ ]:
# drop 'Belgium/Luxembourg'
concat_consumption.drop(axis= 0,labels=[2], inplace=True)

In [ ]:
# rename a whole bunch of entries with strange blank spaces that do not match the names of countries in the gdp dataframe
concat_consumption['Country'].replace({'Viet Nam' : "Vietnam"}, inplace=True)
concat_consumption['Country'].replace({"Côte d'Ivoire" : "Cote d'Ivoire"}, inplace=True)
concat_consumption['Country'].replace({'Bolivia (Plurinational State of)' : "Bolivia"}, inplace=True)
concat_consumption['Country'].replace({"Lao People's Democratic Republic" : "Lao PDR"}, inplace=True)
concat_consumption['Country'].replace({'   Austria' : "Austria"}, inplace=True)
concat_consumption['Country'].replace({"   Belgium" : "Belgium"}, inplace=True)
concat_consumption['Country'].replace({'   Bulgaria' : "Bulgaria"}, inplace=True)
concat_consumption['Country'].replace({"   Croatia" : "Croatia"}, inplace=True)
concat_consumption['Country'].replace({'   Cyprus' : "Cyprus"}, inplace=True)
concat_consumption['Country'].replace({"   Czech Republic" : "Czech Republic"}, inplace=True)
concat_consumption['Country'].replace({'   Denmark' : "Denmark"}, inplace=True)
concat_consumption['Country'].replace({"   Estonia" : "Estonia"}, inplace=True)
concat_consumption['Country'].replace({'   Finland' : "Finland"}, inplace=True)
concat_consumption['Country'].replace({"   France" : "France"}, inplace=True)
concat_consumption['Country'].replace({'   Germany' : "Germany"}, inplace=True)
concat_consumption['Country'].replace({"   Greece" : "Greece"}, inplace=True)
concat_consumption['Country'].replace({'   Hungary' : "Hungary"}, inplace=True)
concat_consumption['Country'].replace({"   Ireland" : "Ireland"}, inplace=True)
concat_consumption['Country'].replace({'   Italy' : "Italy"}, inplace=True)
concat_consumption['Country'].replace({"   Latvia" : "Latvia"}, inplace=True)
concat_consumption['Country'].replace({'   Lithuania' : "Lithuania"}, inplace=True)
concat_consumption['Country'].replace({"   Luxembourg" : "Luxembourg"}, inplace=True)
concat_consumption['Country'].replace({'   Malta' : "Malta"}, inplace=True)
concat_consumption['Country'].replace({"   Netherlands" : "Netherlands"}, inplace=True)
concat_consumption['Country'].replace({'   Poland' : "Poland"}, inplace=True)
concat_consumption['Country'].replace({"   Portugal" : "Portugal"}, inplace=True)
concat_consumption['Country'].replace({'   Romania' : "Romania"}, inplace=True)
concat_consumption['Country'].replace({"   Slovakia" : "Slovakia"}, inplace=True)
concat_consumption['Country'].replace({'   Slovenia' : "Slovenia"}, inplace=True)
concat_consumption['Country'].replace({"   Spain" : "Spain"}, inplace=True)
concat_consumption['Country'].replace({'   Sweden' : "Sweden"}, inplace=True)
concat_consumption['Country'].replace({"   United Kingdom" : "United Kingdom"}, inplace=True)


In [ ]:
# rename column 'country name' to 'country'
gdp_renamed = gdp_data.rename(columns={"Country Name" : "Country"})
gdp_renamed

In [ ]:
# replace name of country 'United States' with 'USA'
gdp_renamed['Country'].replace({'United States':'USA'}, inplace=True)

In [ ]:
# drop unnecessary years and columns
gdp_renamed.drop(['Code','1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969','1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '2019', '2020', 'Unnamed: 65'], axis=1, inplace=True)
gdp_renamed

In [ ]:
# renaming more strangely named countries
gdp_renamed['Country'].replace({'Trinidad and Tobago' : "Trinidad & Tobago"}, inplace=True)
gdp_renamed['Country'].replace({"Venezuela, RB" : "Venezuela"}, inplace=True)
gdp_renamed['Country'].replace({'Slovak Republic' : "Slovakia"}, inplace=True)
gdp_renamed['Country'].replace({"Congo, Rep." : "Congo"}, inplace=True)
gdp_renamed['Country'].replace({'Congo, Dem. Rep.' : "Democratic Republic of Congo"}, inplace=True)
gdp_renamed['Country'].replace({"Yemen, Rep." : "Yemen"}, inplace=True)


In [ ]:
# dropping all entries for countries not found in the 'concat_consumption' dataframe
gdp_trimmed = gdp_renamed[gdp_renamed.Country.isin(concat_consumption.Country)]
gdp_trimmed

In [ ]:
gdp_trimmed.set_index('Country')

In [ ]:
concat_consumption.set_index('Country')

# Load:
Append info from cleaned dataframes to tables in `coffeegdp_db` database

In [ ]:
# save cleaned datasets to 'cleaned-datasets' directory
gdp_trimmed.to_csv('cleaned-datasets/trimmed_gdp.csv', index=False)
concat_consumption.to_csv('cleaned-datasets/concat_consumption.csv', index=False)

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# append entries of 'concat_consumption' df to 'concat_consumption' table in database (this code will NOT run again and create duplicates if it has already been run once)
concat_consumption.to_sql(name='concat_consumption', con=engine, if_exists='append', index=False)

In [ ]:
# append entries of 'gdp_trimmed' df to 'trimmed_gdp' table in database (this code will NOT run again and create duplicates if it has already been run once)
gdp_trimmed.to_sql(name='trimmed_gdp', con=engine, if_exists='append', index=False)